# ООП. Часть 2

## Статические методы


для функций, которые не относятся к конкретным экземплярам класса

- можем использовать для логического объединения функций
- не принимает в себя self! (ссылку на экземпляр класса)
- искользуются как вспомогательные функции, которые работают с данными, которые в них передают в качестве агрументов
- не могут влиять на состояние класса или экземпляра

In [12]:
class Person:
    
    @staticmethod
    def is_adult(age):
        return age >= 18


In [13]:
age = 17
print(Person.is_adult(age))

False


## Методы класса

- принимают в себя класс cls
- метод привязан к классу, а не к экземпляру
- могут менять состояние класса, а это в свою очередь может отразиться на всех объектах, но не могут менять конкретный объект

In [17]:
from datetime import date


class Person:
    
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    @classmethod
    def from_birth_year(cls, name, year):
        return cls(name, date.today().year - year)
    
    @staticmethod
    def is_adult(age):
        return age > 18
    
person1 = Person('Sarah', 25)
person2 = Person.from_birth_year('Roark', 1994)

In [18]:
person2.age

29

### Паттерн singleton

In [21]:
class Singleton:
    def __new__(cls): # new управляет созданием нового экземпляра класса
        if not hasattr(cls, 'instance'):
            cls.instance = super(Singleton, cls).__new__(cls)
        return cls.instance

singleton = Singleton()
new_singleton = Singleton()

In [22]:
singleton = Singleton()

In [23]:
new_singleton = Singleton()

In [24]:
singleton == new_singleton

True

## Property

когда хотим управлять доступом к атрибутам класса

то есть обеспечивает интерфейс для атрибуетов экземпляра класса

In [35]:
class User:
    def __init__(self, name, password):
        self.name = name
        self.password = password # тут вызываем password.setter
        
    @property
    def password(self):
        return f"{self.__password.decode()[:2]}...{self.__password.decode()[-2:]}"
    
    @password.setter
    def password(self, password):
        if len(password) < 8: # тут может быть логика по верификации
            raise Exception("Недостаточная длина пароля.")
        self.__password = password.encode()
    

In [36]:
user = User(name="Lena", password="1234")

Exception: Недостаточная длина пароля.

In [37]:
user = User(name="Lena", password="12345678")

In [38]:
user.name

'Lena'

In [39]:
user.password

'12...78'

In [40]:
user._User__password

b'12345678'

Задание: когда используем classmethod, а когда staticmethod?

Задание: написать класс Animal, у которого есть name, animal_type и voice (то что говорит животное) - animal_type может быть только dog или cat, voice по дефолту None

добавить в этот класс метод, который умеет создавать животное с нужным voice, основываясь на его типе

сделать так чтобы мы могли создать только 1 животное с определенным типом (одного кота, одну собачку)

написать метод, который выводит на экран список допустимых животных

## Метаклассы - фабрики классов

Зачем?
- можно динамически создавать классы, то есть автоматически изменять классы во время создания (например, мы хотим чтобы все классы в модуле или атрибуты в верхнем регистре)
- возвращают классы
- часто используются для реализации API типа Django
- редко используется в обычной практике
- могут создаваться при помощи функции type


[статья](https://habr.com/ru/articles/145835/)

In [41]:
MyDymanicClass = type("MyDymanicClass", (), {})

In [43]:
type(MyDymanicClass)

type

In [44]:
MyDymanicClass = type("MyDymanicClass", (), {"attribute_example": "example"})

In [45]:
my_dynamic_instance = MyDymanicClass()

In [46]:
type(my_dynamic_instance)

__main__.MyDymanicClass

In [47]:
my_dynamic_instance.attribute_example

'example'

Как это делается в реальной жизни

In [ ]:
class Foo:
    __metaclass__ = something...    
    [...]

## Дескрипторы 

[статья](https://pythonist.ru/deskriptory-v-python/)

[руководство по дескрипторам](https://habr.com/ru/articles/122082/)

## MRO - method resolution order

порядок разрешения методов (и атрибутов)

[порядок разрешения методов в python](https://habr.com/ru/articles/62203/)

[и еще одна, возможно более понятная](https://tirinox.ru/mro-python/)

In [50]:
class Employee:    
    @property
    def responsibilities(self):
        return ["ходить в офис", "предоставлять отчеты", "работать с клиентами"]
    
    
class Parent:
    @property
    def responsibilities(self):
        return ["кормить ребенка", "следить за здоровьем", "играть", "развивать"]
    

class Person(Employee, Parent): # какой тут приоритет?
    pass

In [51]:
person = Person()

In [ ]:
person.responsibilities

In [54]:
Person.__mro__

(__main__.Person, __main__.Employee, __main__.Parent, object)

In [56]:
class Employee:    
    @property
    def responsibilities(self):
        return ["ходить в офис", "предоставлять отчеты", "работать с клиентами"]
    
    
class Parent:
    @property
    def responsibilities(self):
        return ["кормить ребенка", "следить за здоровьем", "играть", "развивать"]
    
    @property
    def featues(self):
        return ["есть маленький человек, который похож на тебя"]
    

class Person(Employee, Parent):
    pass

In [57]:
person = Person()
person.featues

['есть маленький человек, который похож на тебя']

Задание: как явно вызвать родительский метод?

## Mixin - классы-примеси

[пример](https://www.codecamp.ru/blog/python-mixins/)

- предоставить множество дополнительных функций для класса;
- использовать определённую функцию во множестве разных классов.

In [12]:
class YandexMusicMixin:
    
    def play_yandex_music(self):
        print("сейчас играет подборка дня")
        
        
class Station:
    def __init__(self):
        self.is_off = True
    
    def turn_on(self):
        self.is_off = False
        
    def turn_off(self):
        self.is_off = True
    
    
class YandexStation(Station, YandexMusicMixin):
    pass


class Phone(YandexMusicMixin):
    def __init__(self, model):
        self.model = model
        
        
class GooglePixel(Phone, YandexMusicMixin):
    pass

In [13]:
yandex_station = YandexStation()
yandex_station.play_yandex_music()

сейчас играет подборка дня


In [14]:
yandex_station.turn_off()

In [15]:
yandex_station.is_off

True

In [16]:
pixel = GooglePixel("pixel 6")
pixel.model

'pixel 6'

In [17]:
pixel.play_yandex_music()

сейчас играет подборка дня


## Абстрактные классы / методы

[ссылка](http://pythonicway.com/education/python-oop-themes/33-python-abstract-class)

- нужны для того чтобы определить интерфейс и сказать что все дочерние классы должны его реализовать
- нельзя создать экземпляр абстрактного класса
- нельзя создать экземпляр класса с нереализованными абстрактными методами

In [1]:
from abc import ABC, abstractmethod

class Animal(ABC):
    def __init__(self, name):
        self.name = name
    
    @abstractmethod
    def make_sound(self):
        pass

In [2]:
class Cat(Animal):
    
    def make_sound(self):
        print("meow")
        
cat = Cat("Kacпер")
cat.make_sound()
cat.name

meow


'Kacпер'

In [3]:
animal = Animal("random animal")

TypeError: Can't instantiate abstract class Animal with abstract method make_sound

In [4]:
class Dog(Animal):
    pass

dog = Dog("Кексик")

TypeError: Can't instantiate abstract class Dog with abstract method make_sound

Можно самостоятельно реализовать:

In [7]:
class Animal:
    def __init__(self, name):
        self.name = name
    
    def make_sound(self):
        raise NotImplementedError("метод make_sound должен быть реализован в дочерних классах!")
        
class Dog(Animal):
    pass

dog = Dog("Кексик")

In [8]:
dog.make_sound()

NotImplementedError: метод make_sound должен быть реализован в дочерних классах!

## Менеджер контекста - with


- можно написать свой, для этого нужно реализовать методы __ enter __ , __ exit __ 
- нужны для того чтобы выделять и освобождать ресурсы строго по необходимости (и не забыть что-то открыть или закрыть)

[почитать](https://pavel-karateev.gitbook.io/intermediate-python/sintaksis/context_managers)

In [10]:
class File:
    def __init__(self, file_name, method):
        self.file_obj = open(file_name, method)
        
    def __enter__(self):
            return self.file_obj
        
    def __exit__(self, type, value, traceback):
        self.file_obj.close()

In [11]:
with File('demo.txt', 'w') as opened_file:
    opened_file.write('Hola!')